In [1]:
import os,sys
import numpy as np
import pandas as pd
# from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import gc

print_to_file = False 
test_run = False 

train_path = '../../../../../../shiyi/homecreditdata/fe_selected/op5_train_fold3_added_fesel.pkl'
test_path = '../../../../../../shiyi/homecreditdata/fe_selected/op5_test_fold3_added_fesel.pkl'

# train = pd.read_pickle(ENV.lightgbm_train_764.value)
train = pd.read_pickle(train_path)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(ENV.lightgbm_test_764.value)
test = pd.read_pickle(test_path)
print('test shape is: {}'.format(test.shape))
fe_id = 'comb_764'

train shape is: (307511, 912)
test shape is: (48744, 911)


In [2]:
train_path = '../../../../../../shiyi/homecreditdata/fe_selected/op5_train_fold3_added_fesel-fillNA.pkl'
test_path = '../../../../../../shiyi/homecreditdata/fe_selected/op5_test_fold3_added_fesel-fillNA.pkl'

In [3]:
train['SK_ID_CURR'] = train['SK_ID_CURR'].astype(int)
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype(int)
targets = train.TARGET.values

train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

X = pd.concat([train.drop('TARGET',axis=1),test])

In [4]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [5]:
def fill_na_list(X,th=200):
    re = scan_nan_portion(X)
    re = re[re<0.15]
    re = re.sort_values()
    col_list = []
    na_list = []
    for col in re.index:
        if re[col] > 0 and X[col].nunique() >= th and str(X[col].dtypes)!= 'category' and str(X[col].dtypes)!= 'object':
            col_list.append(col)
            na_list.append(re[col])
    return col_list,na_list
            
    
def trainLGB(na_train_x,na_val_x,na_train_y,na_val_y):
    reg = LGBMRegressor(n_jobs=30,
                        device='gpu',
                        gpu_device_id=0,
                    boosting_type='gbdt',
                    num_leaves=16,
                    max_depth=-1,
                    learning_rate=0.1,
                    n_estimators=5000,
                    colsample_bytree=0.6,
                    subsample=0.6,
                    silent=-1,
                    verbose=-1, )
    reg.fit(na_train_x, na_train_y, eval_set=[(na_train_x,na_train_y),(na_val_x, na_val_y)],  verbose=200,early_stopping_rounds=100,eval_metric='l1' )
    return reg


def convert_category(X,th=40):
    for col in tqdm(X.columns):
        if X[col].nunique()<th:
            X[col] = X[col].astype('category')
    return X
            
def saving(X):
    print('saving...')
    train_save = X.iloc[:307511].copy()
    train_save['TARGET'] = targets
    print(train_save.shape)

    test_save = X.iloc[307511:].copy()
    train_save.to_pickle(train_path)
    test_save.to_pickle(test_path)

In [6]:
X = convert_category(X)
col_list,na_list = fill_na_list(X)

100%|██████████| 911/911 [00:15<00:00, 58.53it/s]


# CPU

In [8]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col) )
val = 0.1
save_count = 1
count = 0
for col,nav in tqdm(zip(col_list,na_list)):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    seed = 19
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainLGB(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    del reg
    gc.collect()
    

0it [00:00, ?it/s]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_DAYS_BIRTH_min_diff 
the na is:5.613956295350241e-06
(356253, 909)
(2, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 17.6356	training's l2: 536.905	valid_1's l1: 17.8211	valid_1's l2: 576.971
[400]	training's l1: 14.7779	training's l2: 369.308	valid_1's l1: 15.0581	valid_1's l2: 426.526
[600]	training's l1: 13.114	training's l2: 286.603	valid_1's l1: 13.4734	valid_1's l2: 352.078
[800]	training's l1: 11.5072	training's l2: 218.735	valid_1's l1: 11.9236	valid_1's l2: 287.982
[1000]	training's l1: 10.6859	training's l2: 188.103	valid_1's l1: 11.182	valid_1's l2: 261.597
[1200]	training's l1: 9.89211	training's l2: 160.556	valid_1's l1: 10.4509	valid_1's l2: 237.297
[1400]	training's l1: 9.16864	training's l2: 137.586	valid_1's l1: 9.79067	valid_1's l2: 216.176
[1600]	training's l1: 8.71029	training's l2: 123.902	valid_1's l1: 9.40482	valid_1's l2: 205.577
[1800]	training's l1: 8.24238	train

1it [04:14, 254.38s/it]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_AMT_CREDIT_mean_diff 
the na is:5.613956295350241e-06


KeyboardInterrupt: 

# CPU 2

In [7]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col) )
val = 0.1
save_count = 1
count = 0
for col,nav in tqdm(zip(col_list,na_list)):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    seed = 19
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainLGB(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    del reg
    gc.collect()
    

0it [00:00, ?it/s]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_DAYS_BIRTH_min_diff 
the na is:5.613956295350241e-06


KeyboardInterrupt: 

# GPU1

In [7]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col) )
val = 0.1
save_count = 1
count = 0
for col,nav in tqdm(zip(col_list,na_list)):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    seed = 19
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainLGB(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    del reg
    gc.collect()
    

0it [00:00, ?it/s]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_DAYS_BIRTH_min_diff 
the na is:5.613956295350241e-06
(356253, 909)
(2, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 16.4801	training's l2: 458.847	valid_1's l1: 16.5561	valid_1's l2: 466.414
[400]	training's l1: 14.342	training's l2: 331.832	valid_1's l1: 14.4896	valid_1's l2: 355.836
[600]	training's l1: 12.6692	training's l2: 255.454	valid_1's l1: 12.8887	valid_1's l2: 287.123
[800]	training's l1: 11.4711	training's l2: 207.983	valid_1's l1: 11.7631	valid_1's l2: 244.817
[1000]	training's l1: 10.6962	training's l2: 180.348	valid_1's l1: 11.0606	valid_1's l2: 221.577
[1200]	training's l1: 10.0315	training's l2: 158.468	valid_1's l1: 10.4834	valid_1's l2: 203.964
[1400]	training's l1: 9.44534	training's l2: 140.867	valid_1's l1: 9.96758	valid_1's l2: 189.265
[1600]	training's l1: 8.9988	training's l2: 127.571	valid_1's l1: 9.60662	valid_1's l2: 179.46
[1800]	training's l1: 8.6346	trainin

1it [02:35, 155.34s/it]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_AMT_CREDIT_mean_diff 
the na is:5.613956295350241e-06
(356253, 909)
(2, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l1: 3024.71	training's l2: 2.08809e+07	valid_1's l1: 3074.94	valid_1's l2: 2.72192e+07
[400]	training's l1: 2280.46	training's l2: 1.22214e+07	valid_1's l1: 2342.63	valid_1's l2: 1.80972e+07
[600]	training's l1: 1876.68	training's l2: 8.5588e+06	valid_1's l1: 1946.25	valid_1's l2: 1.43713e+07
[800]	training's l1: 1611.56	training's l2: 6.47425e+06	valid_1's l1: 1693.37	valid_1's l2: 1.21949e+07
[1000]	training's l1: 1429	training's l2: 5.15579e+06	valid_1's l1: 1521.53	valid_1's l2: 1.07441e+07
[1200]	training's l1: 1297.9	training's l2: 4.26524e+06	valid_1's l1: 1402.09	valid_1's l2: 9.87859e+06
[1400]	training's l1: 1197.81	training's l2: 3.66727e+06	valid_1's l1: 1316.71	valid_1's l2: 9.36545e+06
[1600]	training's l1: 1099.93	training's l2: 3.13296e+06	valid_1's l1: 1228.61

KeyboardInterrupt: 

# gpu 2

In [8]:
Processed_list = []
ignore_col = ['SK_ID_CURR']
feature_col_base = list(set(X.columns) - set(ignore_col) )
val = 0.1
save_count = 1
count = 0
for col,nav in tqdm(zip(col_list,na_list)):
    print('start processing {} \nthe na is:{}'.format(col,nav))
    X[col] = X[col].replace(365243.0,0)
    X[col] = X[col].replace(np.NINF,np.NAN)
    X[col] = X[col].replace(np.PINF,np.NAN)
    X[col] = X[col].replace(np.Inf,np.NAN)
    feature_col = list(set(feature_col_base) - set([col]))
    
    na_train = X[X[col].notnull()][feature_col].copy()
    na_target = X[X[col].notnull()][col].copy()
    seed = np.random.randint(200) + 1
    seed = 19
    na_train_x,na_val_x,na_train_y,na_val_y = train_test_split(na_train,na_target, test_size=val,random_state=seed)
    print(na_train.shape)
    na_test = X[X[col].isnull()][feature_col].copy()
    print(na_test.shape)
    reg = trainLGB(na_train_x,na_val_x,na_train_y,na_val_y)
    X.loc[X[col].isnull(),col] = reg.predict(na_test)
    Processed_list.append(col)
    print('processed: {}/{}'.format(len(Processed_list),len(col_list)))
    count +=1
    if count % save_count == 0:
        saving(X)
    del reg
    gc.collect()
    

0it [00:00, ?it/s]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_DAYS_BIRTH_min_diff 
the na is:5.613956295350241e-06
(356253, 909)
(2, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 537.133	training's l1: 17.7138	valid_1's l2: 560.501	valid_1's l1: 17.8466
[400]	training's l2: 371.447	training's l1: 14.9301	valid_1's l2: 417.725	valid_1's l1: 15.1771
[600]	training's l2: 281.933	training's l1: 13.0805	valid_1's l2: 338.367	valid_1's l1: 13.3966
[800]	training's l2: 217.817	training's l1: 11.5175	valid_1's l2: 279.601	valid_1's l1: 11.9085
[1000]	training's l2: 177.741	training's l1: 10.4062	valid_1's l2: 243.66	valid_1's l1: 10.8705
[1200]	training's l2: 154.312	training's l1: 9.70725	valid_1's l2: 223.846	valid_1's l1: 10.2426
[1400]	training's l2: 133.609	training's l1: 9.05993	valid_1's l2: 206.45	valid_1's l1: 9.67142
[1600]	training's l2: 120.124	training's l1: 8.60024	valid_1's l2: 195.658	valid_1's l1: 9.28815
[1800]	training's l2: 107.935	train

1it [02:01, 121.62s/it]

start processing NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_AMT_CREDIT_mean_diff 
the na is:5.613956295350241e-06
(356253, 909)
(2, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 2.11252e+07	training's l1: 3041.54	valid_1's l2: 2.57412e+07	valid_1's l1: 3102.51
[400]	training's l2: 1.19626e+07	training's l1: 2234.57	valid_1's l2: 1.61357e+07	valid_1's l1: 2304.32
[600]	training's l2: 8.64703e+06	training's l1: 1907.82	valid_1's l2: 1.22163e+07	valid_1's l1: 1982.27
[800]	training's l2: 6.61294e+06	training's l1: 1642.18	valid_1's l2: 1.02776e+07	valid_1's l1: 1729.3
[1000]	training's l2: 5.36056e+06	training's l1: 1474.68	valid_1's l2: 9.24633e+06	valid_1's l1: 1579.41
[1200]	training's l2: 4.52146e+06	training's l1: 1343.68	valid_1's l2: 8.55201e+06	valid_1's l1: 1463.65
[1400]	training's l2: 3.89723e+06	training's l1: 1240.78	valid_1's l2: 8.03233e+06	valid_1's l1: 1373.64
[1600]	training's l2: 3.39614e+06	training's l1: 1156	valid_1's l2: 7.6254

3it [06:05, 121.70s/it]

...

325it [7:34:53, 83.98s/it]

start processing all_installment_NUM_INSTALMENT_VERSION_std 
the na is:0.049515094524989124
(338615, 909)
(17640, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000383384	training's l1: 0.00551839	valid_1's l2: 0.00939869	valid_1's l1: 0.00672542
[400]	training's l2: 0.000172244	training's l1: 0.00403636	valid_1's l2: 0.00854437	valid_1's l1: 0.00542042
[600]	training's l2: 9.32766e-05	training's l1: 0.0032975	valid_1's l2: 0.00824214	valid_1's l1: 0.00478581
Early stopping, best iteration is:
[674]	training's l2: 7.56534e-05	training's l1: 0.00311467	valid_1's l2: 0.00821799	valid_1's l1: 0.00464292
processed: 326/594
saving...
(307511, 912)


326it [7:35:26, 83.82s/it]

start processing last_20by100_fraction_NUM_INSTALMENT_VERSION_std 
the na is:0.049515094524989124
(338615, 909)
(17640, 909)
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000668387	training's l1: 0.0115604	valid_1's l2: 0.000733501	valid_1's l1: 0.0119398
[400]	training's l2: 0.000375904	training's l1: 0.00875207	valid_1's l2: 0.00045591	valid_1's l1: 0.00933019
[600]	training's l2: 0.000263683	training's l1: 0.00740426	valid_1's l2: 0.000352474	valid_1's l1: 0.00813894
[800]	training's l2: 0.000204477	training's l1: 0.00656001	valid_1's l2: 0.000301134	valid_1's l1: 0.00745358
[1000]	training's l2: 0.000166381	training's l1: 0.00597021	valid_1's l2: 0.000269487	valid_1's l1: 0.00701739
[1200]	training's l2: 0.000138843	training's l1: 0.00548026	valid_1's l2: 0.000247009	valid_1's l1: 0.00666012
[1400]	training's l2: 0.000119288	training's l1: 0.00511006	valid_1's l2: 0.000233143	valid_1's l1: 0.00642946
[1600]	training's l2: 0.000104136	training

KeyboardInterrupt: 